<a href="https://colab.research.google.com/github/Saloni1707/Car_Racing/blob/main/Car_spec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import math
import random
from dataclasses import dataclass
from typing import List,Tuple,Optional
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle,Circle
import matplotlib.animation as animation
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal
from collections import deque
import time

In [ ]:
#basic math
@dataclass
class Vector2D:
  x:float
  y:float
  def __add__(self,other):
    return Vector2D(self.x+other.x,self.y+other.y)
  def __sub__(self,other):
    return Vector2D(self.x-other.x,self.y-other.y)
  def __mul__(self,scalar):
    return Vector2D(self.x*scalar,self.y*scalar)
  def magnitude(self):
    return math.sqrt(self.x**2+self.y**2)
  def normalize(self):
    mag = self.magnitude()
    if mag == 0:
      return Vector2D(0,0)
    return Vector2D(self.x/mag,self.y/mag)
  def rotate(self,angle_rad:float):
    cos_a=math.cos(angle_rad)
    sin_a=math.sin(angle_rad)
    return Vector2D(
        self.x*cos_a-self.y*sin_a,
        self.x*sin_a + self.y*cos_a
    )
    def to_array(self):
      return np.array([self.x,self.y])
print("Vector Math Test")
v1=Vector2D(1,0)
v2=v1.rotate(math.pi/2)
print(f"Orginal:({v1.x:.2f},{v1.y:.2f})")
print(f"Rotated:({v2.x:.2f},{v2.y:.2f})")
print(f"Magnitude: {v1.magnitude():.2f}")


Vector Math Test
Orginal:(1.00,0.00)
Rotated:(0.00,1.00)
Magnitude: 1.00


In [ ]:
#car physics here
@dataclass
class Car:
  def __init__(self,x:float,y:float,angle:float):
    self.position=Vector2D(x,y)
    self.angle=angle
    self.velocity=Vector2D(0,0)
    self.speed=0
    self.angular_velocity=0
    self.max_speed = 50.0 #pixels/second
    self.max_reverse_speed=-20.0
    self.acceleration=30.0
    self.deceleration=50.0
    self.turn_speed=3.0
    self.width=20
    self.length=40
    self.sensor_ranges=[]

  def update(self,dt:float,throttle:float,steering:float):
    throttle=max(-1,min(1,throttle))
    steering=max(-1,min(1,steering))
    if throttle>0: #here based on throttle inc speed
      self.speed+=self.acceleration*throttle*dt
      self.speed=min(self.speed,self.max_speed)
    elif throttle<0:
      self.speed+=self.acceleration*throttle*dt
      self.speed=max(self.speed,self.max_reverse_speed)
    else:
      if self.speed>0:
        self.speed=max(0,self.speed-self.deceleration*dt)
      else:
        self.speed=min(0,self.speed+self.deceleration*dt)

    if abs(self.speed) > 0.1:  # Only turn when moving
        self.angular_velocity = steering * self.turn_speed * (self.speed / self.max_speed)
        self.angle += self.angular_velocity * dt
        self.angle = self.angle % (2 * math.pi)  # Keep angle in [0, 2π]
    direction = Vector2D(math.cos(self.angle), math.sin(self.angle))
    velocity = direction * self.speed
    self.position = self.position + velocity * dt
    self.velocity = velocity

  def get_state(self):
    return np.array([
        self.position.x,self.position.y,
        self.velocity.x,self.velocity.y,
        self.angle,self.angular_velocity,
        self.speed
    ])
  def reset(self,x:float,y:float,angle:float):
    self.position=Vector2D(x,y)
    self.angle=angle
    self.velocity=Vector2D(0,0)
    self.speed=0
    self.angular_velocity=0

print("Car Simulation")
my_car = Car(0, 0, 0)
print(f"Initial State: {my_car.get_state()}")
for i in range(10):
    my_car.update(dt=0.1, throttle=0.5, steering=0)  # Half throttle, no steering

print(f"After 1s straight: Position({my_car.position.x:.1f}, {my_car.position.y:.1f}), Speed: {my_car.speed:.1f}")

for i in range(10):
    my_car.update(dt=0.1, throttle=0.5, steering=0.5)

print(f"After turning: Position({my_car.position.x:.1f}, {my_car.position.y:.1f}), Angle: {math.degrees(my_car.angle):.1f}°")
print()

Car Simulation
Initial State: [0 0 0 0 0 0 0]
After 1s straight: Position(8.2, 0.0), Speed: 15.0
After turning: Position(29.3, 8.5), Angle: 40.0°



In [ ]:
#Our environment
@dataclass
class Environment:
  def __init__(self,width:int,height:int):
    self.width=width
    self.height=height
    self.obstacles=[]
    self.boundary=[]
    self.create_track()

  def create_track(self):
    margin=50
    self.boundary=[
        {"start":Vector2D(0,0),"end":Vector2D(self.width,0)}, #outer boundary top wall
        {"start":Vector2D(0,self.height),"end":Vector2D(self.width,self.height)}, #bottom wall
        {"start":Vector2D(0,0),"end":Vector2D(0,self.height)},#left wall
        {"start":Vector2D(self.width,0),"end":Vector2D(self.width,self.height)},#right wall

        {'start': Vector2D(margin, margin), 'end': Vector2D(self.width-margin, margin)},#Inner boundary box
        {'start': Vector2D(margin, self.height-margin), 'end': Vector2D(self.width-margin, self.height-margin)},
        {'start': Vector2D(margin, margin), 'end': Vector2D(margin, self.height-margin)},
        {'start': Vector2D(self.width-margin, margin), 'end': Vector2D(self.width-margin, self.height-margin)},

    ]


  def is_collision(self,position:Vector2D,car_width:float=20,car_length=40):
    margin=max(car_width,car_length)/2
    if(position.x<margin or position.x>self.width-margin or position.y<margin or position.y>self.height-margin):
      return True
    return False

  def get_track_distance(self,position:Vector2D):
    distances=[]
    distances.append(position.x)
    distances.append(self.width-position.x)
    distances.append(position.y)
    distances.append(self.height-position.y)
    return min(distances)

  def get_start_position(self):
    margin=50
    start_x=margin+30
    start_y=(self.height)//2
    start_angle=0
    start_position=Vector2D(start_x,start_y)
    return start_position,start_angle

  def get_track_boundaries(self):
    margin=50
    left_boundary=[
        Vector2D(0,0),
        Vector2D(0,self.height),
        Vector2D(margin,self.height-margin),
        Vector2D(margin,margin),
        Vector2D(0,0)
    ]
    right_boundary = [
        Vector2D(self.width, 0),
        Vector2D(self.width, self.height),
        Vector2D(self.width - margin, self.height - margin),
        Vector2D(self.width - margin, margin),
        Vector2D(self.width, 0)
    ]
    return left_boundary,right_boundary


print("Test Environment")
env=Environment(width=800, height=600)
test_pos=Vector2D(100,100)
print(f"Position ({test_pos.x}, {test_pos.y}):")
print(f"Collision: {env.is_collision(test_pos)}")
print(f"Distance to boundary: {env.get_track_distance(test_pos):.1f}")
print()

Test Environment
Position (100, 100):
Collision: False
Distance to boundary: 100.0



In [ ]:
class CarWithSensors(Car):
  def __init__(self,x:float=0,y:float=0,angle:float=0):
    super().__init__(x,y,angle)
    self.num_sensors=8
    self.sensor_range=100
    self.sensor_angles=[]
    self.sensor_readings=[]

    for i in range(self.num_sensors):
      angle_offset = (i-self.num_sensors/2 + 0.5)*(180/self.num_sensors) #this is the sensor in front of car
      self.sensor_angles.append(math.radians(angle_offset))
    print("This is the angle",self.sensor_angles)

  def update_sensors(self,Environment):
    self.sensor_readings=[]#Based on our env we update sensor readings here

    for sensor_angle in self.sensor_angles:
      world_angle=self.angle+sensor_angle
      ray_dx=math.cos(world_angle)
      ray_dy=math.sin(world_angle)#ray directions of our sensors

      distance=self.cast_ray(ray_dx,ray_dy,Environment)
      self.sensor_readings.append(distance)

  def cast_ray(self,dx:float,dy:float,Environment):
    step_size=2
    max_steps=int(self.sensor_range/step_size)

    for step in range(max_steps):
      ray_x=self.position.x+dx*step*step_size
      ray_y=self.position.y+dy*step*step_size
      ray_pos = Vector2D(ray_x,ray_y) #chk if ray hit the boundary
      if Environment.is_collision(ray_pos,car_width=1):
        return step*step_size

    return self.sensor_range #ray didn't hit anything all good

  def get_sensor_state(self):
    normalized_readings=[reading/self.sensor_range for reading in self.sensor_readings]
    return np.array(normalized_readings)

  def get_full_state(self):
    car_state=super().get_state()
    sensor_state=self.get_sensor_state()
    return np.concatenate([car_state,sensor_state])

print("Sensor with car test")
env=Environment(width=800,height=600)
sensor_car=CarWithSensors(100,300,0)
sensor_car.update_sensors(env)
print(f"Sensor Readings: {sensor_car.sensor_readings}")
print(f"Sensor State: {sensor_car.get_sensor_state()}")


Sensor with car test
This is the angle [-1.3744467859455345, -0.9817477042468103, -0.5890486225480862, -0.19634954084936207, 0.19634954084936207, 0.5890486225480862, 0.9817477042468103, 1.3744467859455345]
Sensor Readings: [100, 100, 100, 100, 100, 100, 100, 100]
Sensor State: [1. 1. 1. 1. 1. 1. 1. 1.]


In [ ]:
class ImprovedDQN(nn.Module):
  def __init__(self,state_size,action_size,hidden_dim=256):
    super(ImprovedDQN,self).__init()
    self.network=nn.sequential(
        nn.Linear(state_size,hidden_dim),
        nn.RELU(),
        nn.Linear(hidden_dim,hidden_dim),
        nn.RELU(),
        nn.Linear(hidden_dim,128),
        nn.RELU(),
        nn.Linear(128,action_size)
    )
    def forward(self,x):
      return self.network(x);

In [ ]:
@dataclass
class DataCollector:
  def __init__(self,max_samples:int=10000):
    self.states=[]
    self.actions=[]
    self.rewards=[]
    self.max_samples=max_samples

  def add_sample(self,state:np.ndarray,action:np.ndarray,reward:float):
    self.states.append(state.copy())
    self.actions.append(action.copy())
    self.rewards.append(reward)

    if(len(self.states)>self.max_samples):
      self.states.pop(0) #sliding window type
      self.actions.pop(0)
      self.rewards.pop(0)

  def get_data(self,batch_size:int=32):
    if(len(self.states)<batch_size):
      return None,None,None

    indices=random.sample(range(len(self.states)),batch_size)
    batch_states=[self.states[i] for i in indices]
    batch_actions=[self.actions[i] for i in indices]
    batch_rewards=[self.rewards[i] for i in indices]

    return(
        torch.FloatTensor(batch_states),
        torch.FloatTensor(batch_actions),
        torch.FloatTensor(batch_rewards)
    )

  def clear(self):
    self.states.clear()
    self.actions.clear()
    self.rewards.clear()

In [ ]:
@dataclass
class RewardSystem:
  def __init__(self):
    self.prev_position=None
    self.prev_distance=None
  def calculate_reward(self,car:Car,env:Environment):
    reward=0
    if not env.is_collision(car.position):
      reward += 1.0
      speed_reward = min(car.speed / car.max_speed,1.0)*0.5 #speed reward
      reward += speed_reward

      distance_track=env.get_track_distance(car.position)
      if distance_track < 20:
        reward += 0.3
      if distance_track > 40:
        reward-=0.2
    else:
      reward = -10.0
    if abs(car.speed) < 1.0:
      reward -=0.1

    return reward